## 映画のレビューより感情分析
### J2200036 小野慧

#### このレポートでは感情分析を行ってみる。使用するデータは、
https://ai.stanford.edu/~amaas/data/sentiment/
#### にあるもので、Andrew Maas氏によって提供されたものだ。trainデータセットには映画のレビューが25,000件含まれており、トレーニング用とテスト用にそれぞれ同じ数が提供されている。testデータにも25,000件のレビューが含まれている。また、生のテキストデータと既に処理されたワードバッグ形式のデータがあるが、今回は生のテキストデータを用いる。
#### このレポートの目的は、trainデータの映画のレビューコメントから、否定的・肯定的なコメントの特徴を学習してモデルを作り、testデータにて否定か肯定かの二値分類を行うことである。

In [1]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses

#### 直接データセットをローカルにダウンロードし、展開するにはかなり時間がかかったので、tf.keras.utils.get_file()という関数を用いて、ページから直接ダウンロードする。この関数は、Kerasのユーティリティ関数の一部であり、TensorFlowの中にあるKerasモジュールにて利用できる。

In [2]:
# データセットのダウンロード
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

In [6]:
# dataset_dirの下部構造を確認

os.listdir(dataset_dir)

['imdbEr.txt', 'imdb.vocab', '.ipynb_checkpoints', 'test', 'train', 'README']

In [7]:
# train_dirの下部構造を確認

train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['labeledBow.feat',
 'neg',
 'unsupBow.feat',
 'urls_pos.txt',
 'pos',
 'urls_neg.txt',
 'urls_unsup.txt',
 'unsup']

In [8]:
# 例として、肯定的なレビューをひとつ確認

sample_file = os.path.join(train_dir, 'pos/1181_9.txt')
with open(sample_file) as f:
    print(f.read())

Rachel Griffiths writes and directs this award winning short film. A heartwarming story about coping with grief and cherishing the memory of those we've loved and lost. Although, only 15 minutes long, Griffiths manages to capture so much emotion and truth onto film in the short space of time. Bud Tingwell gives a touching performance as Will, a widower struggling to cope with his wife's death. Will is confronted by the harsh reality of loneliness and helplessness as he proceeds to take care of Ruth's pet cow, Tulip. The film displays the grief and responsibility one feels for those they have loved and lost. Good cinematography, great direction, and superbly acted. It will bring tears to all those who have lost a loved one, and survived.


#### 学習用のデータセットを作成

In [9]:
# 余分なフォルダを削除

remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [11]:
# trainデータを8:2に分割、うち8割をtrainデータとする

batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='training', 
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [12]:
# trainデータを8:2に分割、うち2割を検証データとする

raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [13]:
# testデータをバッチサイズ32に分割

raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test', 
    batch_size=batch_size)

Found 25000 files belonging to 2 classes.


In [14]:
# テキストの前処理

def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

In [16]:
# テキストをベクトル化するレイヤーを定義

max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [18]:
# ベクトル化レイヤーをトレーニングデータに合わせて調整

train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [20]:
# テキストデータを数値化する関数を定義

def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [22]:
# テキストデータを数値化

text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

Review tf.Tensor(b'Recipe for one of the worst movies of all time: a she-male villain who looks like it escaped from the WWF, has terrible aim with a gun that has inconsistent effects (the first guy she shoots catches on fire but when she shoots anyone else they just disappear) and takes time out to pet a deer. Then you got the unlikable characters, 30 year old college students, a lame attempt at a surprise ending and lots, lots more. Avoid at all costs.', shape=(), dtype=string)
Label neg
Vectorized review (<tf.Tensor: shape=(1, 250), dtype=int64, numpy=
array([[9257,   15,   28,    5,    2,  241,   91,    5,   30,   58,    4,
           1, 1011,   36,  262,   38,    9, 3891,   35,    2,    1,   43,
         382, 5223,   16,    4, 1113,   12,   43, 5739,  300,    2,   83,
         225,   55, 3209, 3898,   20,  973,   18,   51,   55, 3209,  250,
         320,   34,   40, 4386,    3,  294,   58,   44,    6, 2911,    4,
        6757,   92,   22,  184,    2, 4916,  100, 1221,  336,  161, 

In [23]:
# レイヤーを適用

train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [25]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [29]:
# モデルの作成

embedding_dim = 16
model = tf.keras.Sequential([
    layers.Embedding(max_features + 1, embedding_dim),
    layers.Dropout(0.2),
    layers.GlobalAveragePooling1D(),
    layers.Dropout(0.2),
    layers.Dense(1)
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 16)          160016    
                                                                 
 dropout_2 (Dropout)         (None, None, 16)          0         
                                                                 
 global_average_pooling1d_1  (None, 16)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160033 (625.13 KB)
Trainable params: 160033 (625.13 KB)
Non-trainable params: 0 (0.00 Byte)
______________

In [30]:
# 損失関数、オプティマイザ、評価メトリクスの設定

model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [31]:
# モデルのトレーニング

epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/10
625/625 [==============================] - 7s 10ms/step - loss: 0.6625 - binary_accuracy: 0.6989 - val_loss: 0.6136 - val_binary_accuracy: 0.7720
Epoch 2/10
625/625 [==============================] - 4s 6ms/step - loss: 0.5477 - binary_accuracy: 0.8023 - val_loss: 0.4982 - val_binary_accuracy: 0.8250
Epoch 3/10
625/625 [==============================] - 4s 6ms/step - loss: 0.4446 - binary_accuracy: 0.8439 - val_loss: 0.4202 - val_binary_accuracy: 0.8484
Epoch 4/10
625/625 [==============================] - 4s 6ms/step - loss: 0.3783 - binary_accuracy: 0.8668 - val_loss: 0.3737 - val_binary_accuracy: 0.8614
Epoch 5/10
625/625 [==============================] - 4s 6ms/step - loss: 0.3353 - binary_accuracy: 0.8802 - val_loss: 0.3448 - val_binary_accuracy: 0.8678
Epoch 6/10
625/625 [==============================] - 4s 6ms/step - loss: 0.3041 - binary_accuracy: 0.8895 - val_loss: 0.3257 - val_binary_accuracy: 0.8726
Epoch 7/10
625/625 [==============================] - 4s 6ms/st

In [32]:
# モデルの評価

loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

782/782 [==============================] - 2s 2ms/step - loss: 0.3111 - binary_accuracy: 0.8718
Loss:  0.31110289692878723
Accuracy:  0.8717600107192993


#### よって上記の通りに、約87.1%の精度でテキストデータの二値分類予測ができた。

#### 引用
Stanford AI Group Website: https://ai.stanford.edu/~amaas/data/sentiment/

@InProceedings{maas-EtAl:2011:ACL-HLT2011,
  author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher
    title     = {Learning Word Vectors for Sentiment Analysi}  
  booktitle = {Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologi
  
  month     = {J
  ,
  year      = {
  },
  address   = {Portland, Oregon
  A},
  publisher = {Association for Computational Lingu
  cs},
  pages     = {1
  150},
  url       = {http://www.aclwebt#-1015}
}

#### 参考
Tensorflow: https://www.tensorflow.org/tutorials/keras/text_classification?hl=ja-1015}

note: https://note.com/hroy/n/ncb1469281410